# 

title: Water Rights Restored to the Gila River

subtitle: The impacts of irrigation on vegetation health in the Gila
River Valley

author:

-   Elsa Culler

-   Nate Quarderer

date: last-modified

image: /img/earth-analytics/water-rights/lesson-water-rights.png

image-alt: “Dry river with dead plants turns into a stream with living
plants”

description: \|

In 2004, the Akimel O’‘otham and Tohono O’’odham tribes won a water
rights settlement in the US Supreme Court. Using satellite imagery, we
can see the effects of irrigation water on the local vegetation.

learning-goals:

-   Open raster or image data using code

-   Combine raster data and vector data to crop images to an area of
    interest

-   Summarize raster values with stastics

-   Analyze a time-series of raster images

params:

id: stars

site_name: Gila River Indian Community

event: water rights case

data_dir: gila-river

jupyter:

kernelspec:

    name: learning-portal

    language: python

    display_name: Learning Portal

# STEP 2: Wrangle Raster Data

# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [11]:
%store -r boundary_gdf gdf

You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [12]:
# Import libraries

import pandas as pd
from pathlib import Path
import rioxarray
import xarray as xr

## Load in NDVI data

Now you need to load all the downloaded files into Python. Let’s start
by getting all the file names. You will also need to extract the date
from the filename. Check out [the lesson on getting information from
filenames in the
textbook](https://www.earthdatascience.org/courses/intro-to-earth-data-science/write-efficient-python-code/loops/data-workflows-with-loops/).

In this lesson, you will use the `glob.glob()` function in Python to
find all files that match a **pattern** formed with the **wildcard**
character `*`. The wildcard can represent any string of alphanumberic
characters. For example, the pattern `'file_*.tif'` will match the files
`'file_1.tif'`, `'file_2.tiv'`, or even
`'file_qeoiurghtfoqaegbn34pf.tif'`… but it will not match
`'something-else.csv'` or even `'something-else.tif'`.

In this notebook, we’ll use the `.rglob()`, or **recursive** glob method
of the Path object instead. It works similarly, but you’ll notice that
we have to convert the results to a list with the `list()` function.

> **GOTCHA ALERT!**
>
> `glob` doesn’t necessarily find files in the order you would expect.
> Make sure to **sort** your file names like it says in the textbook.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Take a look at the file names for the NDVI files. What do you notice
is the same for all the files? Keep in mind that you only want the NDVI
files, not the Quality files (which will flag potential incorrect
measurements).</p></div></div>

The same product name and site reference, but a different date stamp.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Create a <strong>pattern</strong> for the files you want to import.
Your pattern should include the parts of the file names that are the
same for all files, and replace the rest with the <code>*</code>
character. Make sure to match the NDVI files, but not the Quality
files!</li>
<li>Replace <code>ndvi-pattern</code> with your pattern</li>
<li>Run the code and make sure that you are getting all the files you
want and none of the files you don’t!</li>
</ol></div></div>

In [13]:
# Get a sorted list of NDVI tif file paths
project_dir = Path("/workspaces/data/gilariverdata")
ndvi_paths = sorted(list(project_dir.rglob('*NDVI*.tif')))

# Display the first and last three files paths to check the pattern
ndvi_paths[:3], ndvi_paths[-3:]

([PosixPath('/workspaces/data/gilariverdata/gila-river-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2001145000000_aid0001.tif'),
  PosixPath('/workspaces/data/gilariverdata/gila-river-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2001161000000_aid0001.tif'),
  PosixPath('/workspaces/data/gilariverdata/gila-river-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2001177000000_aid0001.tif')],
 [PosixPath('/workspaces/data/gilariverdata/gila-river-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2022209000000_aid0001.tif'),
  PosixPath('/workspaces/data/gilariverdata/gila-river-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2022225000000_aid0001.tif'),
  PosixPath('/workspaces/data/gilariverdata/gila-river-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2022241000000_aid0001.tif')])

## Repeating tasks in Python

Now you should have a few dozen files! For each file, you need to:

-   Load the file in using the `rioxarray` library
-   Get the date from the file name
-   Add the date as a dimension coordinate
-   Give your data variable a name

You don’t want to write out the code for each file! That’s a recipe for
copy pasta. Luckily, Python has tools for doing similar tasks
repeatedly. In this case, you’ll use one called a `for` loop.

There’s some code below that uses a `for` loop in what is called an
**accumulation pattern** to process each file. That means that you will
save the results of your processing to a list each time you process the
files, and then merge all the arrays in the list.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ul>
<li>Look at the file names. How many characters from the end is the
date? <code>doy_start</code> and <code>doy_end</code> are used to
extract the day of the year (doy) from the file name. You will need to
count characters from the end and change the values to get the right
part of the file name. HINT: the index -1 in Python means the last
value, -2 second-to-last, and so on.</li>
<li>Replace any required variable names with your chosen variable
names</li>
</ul></div></div>

In [14]:
doy_start = -21
doy_end = -14

# Loop through each NDVI image
ndvi_das = []
for ndvi_path in ndvi_paths:
    # Get date from file name
    date_str = ndvi_path.stem[doy_start:doy_end]  
    date = pd.to_datetime(date_str, format="%Y%j")

    # Open dataset
    da = rioxarray.open_rasterio(ndvi_path, masked=True).squeeze(drop=True)

    # Add date dimension and clean up metadata
    da = da.assign_coords({'date': date})
    da = da.expand_dims({'date': 1})
    da.name = 'NDVI'

    # Prepare for concatenation
    ndvi_das.append(da)

## Combine Rasters

Next, stack your arrays by date into a time series using the
`xr.combine_by_coords()` function. You will have to tell it which
dimension you want to stack your data in.

In [15]:
# Combine NDVI images from all dates
ndvi_combined = xr.combine_by_coords(
    ndvi_das,                 # list of DataArrays, each with a 'date' coord and dim
    data_vars="minimal",
    coords="minimal",
    combine_attrs="override"
)

# If combine returns a Dataset, pull out the NDVI variable; otherwise it's already a DataArray
ndvi_xr = ndvi_combined["NDVI"] if isinstance(ndvi_combined, xr.Dataset) else ndvi_combined

# ensure chronological order
ndvi_xr = ndvi_xr.sortby("date")

ndvi_xr

/tmp/ipykernel_65285/1068419180.py:2: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ndvi_combined = xr.combine_by_coords(


<xarray.DataArray 'NDVI' (date: 154, y: 203, x: 382)> Size: 48MB
array([[[8282., 6146., 3796., ..., 1542., 1542., 1774.],
        [5235., 3203., 4636., ..., 1450., 1450., 1450.],
        [5235., 3851., 4567., ...,  962.,  580.,  641.],
        ...,
        [1376., 1376., 1283., ..., 1489., 1489., 1568.],
        [1299., 1299., 1238., ..., 1831., 1831., 2317.],
        [1347., 1309., 1329., ..., 1571., 2384., 2452.]],

       [[5146., 4225., 3706., ..., 1656., 1656., 1656.],
        [3543., 4138., 3706., ..., 1275., 1275.,  968.],
        [4803., 3543., 3687., ...,  750.,  750.,  968.],
        ...,
        [1308., 1308., 1234., ..., 1592., 1592., 1530.],
        [1270., 1270., 1164., ..., 1689., 1689., 2335.],
        [1258., 1179., 1179., ..., 2157., 2549., 1965.]],

       [[6352., 4851., 4630., ..., 1628., 1628., 1591.],
        [7064., 5079., 3888., ...,  830.,  830., 1600.],
        [5084., 4632., 5103., ..., 1243., 1243.,  830.],
        ...,
...
        [1215., 1215., 1133., ..., 1546., 1546., 1628.],
        [1163., 1163., 1114., ..., 2063., 2063., 2372.],
        [1166., 1107., 1109., ..., 1884., 1937., 2372.]],

       [[3886., 3537., 3416., ..., 1927., 1927., 1597.],
        [3795., 3638., 3036., ..., 1491., 1491., 1285.],
        [2813., 3478., 4445., ..., 1157., 1157.,  738.],
        ...,
        [1303., 1303., 1303., ..., 1739., 1739., 1859.],
        [1332., 1332., 1303., ..., 1940., 1940., 2383.],
        [1662., 1332., 1348., ..., 1729., 2034., 2325.]],

       [[5902., 3793., 3145., ..., 1536., 1536., 1564.],
        [6694., 4878., 3291., ..., 1590., 1590., 1331.],
        [3103., 3941., 4527., ..., 1540., 1026.,  448.],
        ...,
        [1338., 1338., 1192., ..., 1740., 1740., 1630.],
        [1289., 1289., 1205., ..., 2146., 2146., 2025.],
        [1307., 1157., 1263., ..., 1736., 2146., 2085.]]],
      shape=(154, 203, 382), dtype=float32)
Coordinates:
  * x            (x) float64 3kB -112.3 -112.3 -112.3 ... -111.5 -111.5 -111.5
  * y            (y) float64 2kB 33.39 33.39 33.38 33.38 ... 32.97 32.97 32.97
    spatial_ref  int64 8B 0
  * date         (date) datetime64[ns] 1kB 2001-05-25 2001-06-10 ... 2022-08-29
Attributes:
    add_offset:     0.0
    scale_factor:   0.0001
    units:          NDVI
    AREA_OR_POINT:  Area

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [16]:
%store ndvi_das ndvi_xr boundary_gdf 

Stored 'ndvi_das' (list)
Stored 'ndvi_xr' (DataArray)
Stored 'boundary_gdf' (GeoDataFrame)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!